In [16]:
# imports
from csv import DictReader, DictWriter
from tqdm import tqdm
import pandas as pd
import numpy as np

In [17]:
original_column_names = ['station_id', 'longitude', 'latitude', 'time', 'AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'WaveHeight', 'WavePeriod', 'MeanWaveDirection', 'Hmax', 'AirTemperature', 'DewPoint', 'SeaTemperature', 'RelativeHumidity']

new_column_names = ['AtmospherePressure', 'WindDirection', 'WindSpeed', 'Gust', 'AirTemperature', 'RelativeHumidity', 'WaveHeight']

In [18]:
with open('../data/raw_data.csv', 'r') as f1, open('../data/interim_data.csv', 'w') as f2:
    reader = DictReader(f1, fieldnames=original_column_names)
    writer = DictWriter(f2, fieldnames=new_column_names, lineterminator='\n')

    next(reader)
    next(reader)

    writer.writeheader()

    for line in tqdm(reader):
        if any(line[column_name] == 'NaN' for column_name in new_column_names):
            continue
        writer.writerow({column_name: line[column_name] for column_name in new_column_names})

613392it [00:02, 284366.26it/s]


In [19]:
df = pd.read_csv('../data/interim_data.csv')
df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,RelativeHumidity,WaveHeight
count,403845.000000,403845.000000,403845.000000,403845.000000,403845.00000,403845.000000,403845.000000
mean,1013.774914,206.758608,15.104633,21.405105,11.61408,82.416231,2.289677
std,12.234193,89.428016,6.875784,9.522440,2.87607,9.714247,1.523549
min,909.900000,0.000000,0.000000,0.000000,0.26400,0.391000,0.078000
25%,1006.400000,150.000000,10.000000,14.658000,9.50000,75.391000,1.200000
50%,1015.000000,220.000000,15.000000,20.000000,11.60000,83.000000,1.953000
75%,1022.290000,271.000000,19.781000,27.000000,14.00000,90.000000,3.000000
max,1046.400000,360.000000,76.506000,200.000000,26.17200,100.000000,14.100000


In [20]:
# Use the interquartile range to remove the outliers in data.
for column_name in new_column_names:
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_whisker = Q1 - 1.5*IQR
    upper_whisker = Q3 + 1.5*IQR
    df = df[(df[column_name] >= lower_whisker) & (df[column_name] <= upper_whisker)]

df.describe()

,AtmospherePressure,WindDirection,WindSpeed,Gust,AirTemperature,RelativeHumidity,WaveHeight
count,378551.000000,378551.000000,378551.000000,378551.000000,378551.000000,378551.000000,378551.000000
mean,1014.768286,204.638287,14.465392,20.374661,11.724571,82.731325,2.057696
std,11.285860,90.320138,6.284241,8.417713,2.874487,9.571322,1.164179
min,982.593000,0.000000,0.000000,0.000000,2.896000,53.516000,0.078000
25%,1007.600000,150.000000,10.000000,14.000000,9.600000,76.000000,1.200000
50%,1015.600000,216.000000,14.000000,20.000000,11.800000,83.594000,1.800000
75%,1022.620000,270.000000,19.000000,25.189000,14.100000,90.234000,2.734000
max,1045.800000,360.000000,33.016000,46.251000,20.771000,100.000000,5.430000


In [21]:
# Feature engineering
azimuth_rad = np.deg2rad(df['WindDirection'])
df['WindDirectionSin'] = np.sin(azimuth_rad)
df['WindDirectionCos'] = np.cos(azimuth_rad)
df = df.drop(columns=['WindDirection'])

df = df[['AtmospherePressure', 'WindSpeed', 'Gust', 'RelativeHumidity', 'WindDirectionSin', 'WindDirectionCos', 'AirTemperature', 'WaveHeight']]

In [22]:
df.to_csv('../data/processed_data.csv', index=False)